In [257]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [258]:
%%sql
drop table if exists Seats;
CREATE TABLE Seats
(seat   INTEGER NOT NULL  PRIMARY KEY,
 status CHAR(2) NOT NULL
 CHECK (status IN ('空', '占')) ); 

INSERT INTO Seats VALUES (1,  '占');
INSERT INTO Seats VALUES (2,  '占');
INSERT INTO Seats VALUES (3,  '空');
INSERT INTO Seats VALUES (4,  '空');
INSERT INTO Seats VALUES (5,  '空');
INSERT INTO Seats VALUES (6,  '占');
INSERT INTO Seats VALUES (7,  '空');
INSERT INTO Seats VALUES (8,  '空');
INSERT INTO Seats VALUES (9,  '空');
INSERT INTO Seats VALUES (10, '空');
INSERT INTO Seats VALUES (11, '空');
INSERT INTO Seats VALUES (12, '占');
INSERT INTO Seats VALUES (13, '占');
INSERT INTO Seats VALUES (14, '空');
INSERT INTO Seats VALUES (15, '空');

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [260]:
%%sql
-- cross joinを用いる方法
select s1.seat
from seats s1
cross join seats s2
where s2.seat = s1.seat + 2
and not exists (
    select *
    from Seats s3
    where s3.seat between s1.seat and s2.seat and
    s3.status <> '空'
)

*  postgresql://padawan:***@db:5432/dsdojo_db
4 rows affected.


seat
3
7
8
9


In [261]:
%%sql
-- havingを用いる方法
select s1.seat
from seats s1
inner join seats s2
on s2.seat between s1.seat and s1.seat + 2
group by s1.seat
having count(*) = 3 and
min(s2.status) = '空' and
max(s2.status) = '空'

*  postgresql://padawan:***@db:5432/dsdojo_db
4 rows affected.


seat
3
7
8
9


In [268]:
%%sql
-- inner joinを用いずやると冗長になる
select seat
from Seats as s1
where exists (
    select 1
    from Seats s2
    where s1.seat <= s2.seat and
    3 >= (
        select count(*)
        from Seats s3
        where s3.seat between s1.seat and s2.seat
    )
    having count(*) = 3 and
    max(status) = '空' and
    min(status) = '空'
)

*  postgresql://padawan:***@db:5432/dsdojo_db
4 rows affected.


seat
3
7
8
9


In [272]:
%%sql
-- 連続するitemが'空'である数が3の場合を求めれば良い(その1)
select seat
from Seats s1
where 3 = (
    select count(*)
    from seats s2
    where s2.status = '空' and
    s2.seat between s1.seat and s1.seat+2
)

*  postgresql://padawan:***@db:5432/dsdojo_db
4 rows affected.


seat
3
7
8
9


In [271]:
%%sql
-- 連続するitemが'空'である数が3の場合を求めれば良い(その2)
select seat
from Seats as s1
where not exists (
    select count(*) as cnt
    from Seats as s2
    where s2.seat between s1.seat and s1.seat+2
    and s2.status = '空'
    having count(*) <> 3
)

*  postgresql://padawan:***@db:5432/dsdojo_db
4 rows affected.


seat
3
7
8
9


In [270]:
%%sql
-- window関数を用いる方法
select seat
from (
    select seat,
        max(seat) over (
            order by seat
            rows between 2 following
            and 2 following
        ) as end_seat
    from seats
    where status = '空'
) as tmp
where end_seat - seat = 2

*  postgresql://padawan:***@db:5432/dsdojo_db
4 rows affected.


seat
3
7
8
9
